In [1]:
pip install ray

     |████████████████████████████████| 54.7 MB 17 kB/s 
     |████████████████████████████████| 118 kB 35.2 MB/s 


In [2]:
pip install xgboost_ray

     |████████████████████████████████| 136 kB 5.6 MB/s 


In [3]:
pip install sklearn

In [4]:
from xgboost_ray import RayDMatrix, RayParams, train
from sklearn.datasets import load_breast_cancer

train_x, train_y = load_breast_cancer(return_X_y=True)
train_set = RayDMatrix(train_x, train_y)

evals_result = {}
bst = train(
    {
        "objective": "binary:logistic",
        "eval_metric": ["logloss", "error"],
    },
    train_set,
    evals_result=evals_result,
    evals=[(train_set, "train")],
    verbose_eval=False,
    ray_params=RayParams(num_actors=2, cpus_per_actor=1))

bst.save_model("model.xgb")
print("Final training error: {:.4f}".format(
    evals_result["train"]["error"][-1]))

/usr/local/lib/python3.7/dist-packages/redis/connection.py:77: UserWarning: redis-py works best with hiredis. Please consider installing
  warnings.warn(msg)
/usr/local/lib/python3.7/dist-packages/xgboost_ray/main.py:166: UserWarning: You are using `xgboost_ray` with a legacy XGBoost version (version 0.90). While we try to support older XGBoost versions, please note that this library is only fully tested and supported for XGBoost >= 1.4. Please consider upgrading your XGBoost version (`pip install -U xgboost`).
  warnings.warn(LEGACY_WARNING)
(raylet) /usr/local/lib/python3.7/dist-packages/redis/connection.py:77: UserWarning: redis-py works best with hiredis. Please consider installing
(raylet)   warnings.warn(msg)
2021-11-16 09:40:21,684	INFO main.py:971 -- [RayXGBoost] Created 2 new actors (2 total actors). Waiting until actors are ready for training.
(raylet) /usr/local/lib/python3.7/dist-packages/redis/connection.py:77: UserWarning: redis-py works best with hiredis. Please consider

Final training error: 0.0053


In [5]:
from xgboost_ray import RayDMatrix, RayParams, predict
from sklearn.datasets import load_breast_cancer
import xgboost as xgb

data, labels = load_breast_cancer(return_X_y=True)

dpred = RayDMatrix(data, labels)

bst = xgb.Booster(model_file="model.xgb")
pred_ray = predict(bst, dpred, ray_params=RayParams(num_actors=2))

print(pred_ray)

/usr/local/lib/python3.7/dist-packages/xgboost_ray/main.py:166: UserWarning: You are using `xgboost_ray` with a legacy XGBoost version (version 0.90). While we try to support older XGBoost versions, please note that this library is only fully tested and supported for XGBoost >= 1.4. Please consider upgrading your XGBoost version (`pip install -U xgboost`).
  warnings.warn(LEGACY_WARNING)
2021-11-16 09:40:37,592	INFO main.py:1535 -- [RayXGBoost] Created 2 remote actors.
(raylet) /usr/local/lib/python3.7/dist-packages/redis/connection.py:77: UserWarning: redis-py works best with hiredis. Please consider installing
(raylet)   warnings.warn(msg)
(raylet) /usr/local/lib/python3.7/dist-packages/redis/connection.py:77: UserWarning: redis-py works best with hiredis. Please consider installing
(raylet)   warnings.warn(msg)
2021-11-16 09:40:40,080	INFO main.py:1552 -- [RayXGBoost] Starting XGBoost prediction.


[0.07511145 0.04715708 0.02642454 0.11464816 0.08779665 0.19609766
 0.02642454 0.03200296 0.03968306 0.06930447 0.09290799 0.02642454
 0.03476055 0.05490047 0.05012454 0.02642454 0.03337562 0.02642454
 0.02642454 0.9528658  0.9751126  0.9751126  0.07511145 0.02642454
 0.02642454 0.04102892 0.03118352 0.02642454 0.02642454 0.08779665
 0.02642454 0.03476055 0.02642454 0.02642454 0.02642454 0.02642454
 0.0665335  0.9690314  0.33619487 0.0696831  0.5729184  0.15662025
 0.03451282 0.03200296 0.12301622 0.04102892 0.9751126  0.04096693
 0.97315884 0.9234187  0.9751126  0.9751126  0.9751126  0.03059936
 0.09855346 0.9751126  0.02642454 0.03755146 0.9751126  0.97315884
 0.9751126  0.97315884 0.02642454 0.9751126  0.03120249 0.03120249
 0.97315884 0.9751126  0.8630876  0.9751126  0.02642454 0.9751126
 0.02642454 0.18486539 0.9751126  0.02642454 0.9684864  0.07511145
 0.04528174 0.9751126  0.97315884 0.8423582  0.02642454 0.02642454
 0.9751126  0.02642454 0.18525673 0.02642454 0.9610139  0.91992

In [6]:
from xgboost_ray import RayDMatrix, RayParams, train
from sklearn.datasets import load_breast_cancer

num_actors = 1
num_cpus_per_actor = 1

ray_params = RayParams(
    num_actors=num_actors, cpus_per_actor=num_cpus_per_actor)

def train_model(config):
    train_x, train_y = load_breast_cancer(return_X_y=True)
    train_set = RayDMatrix(train_x, train_y)

    evals_result = {}
    bst = train(
        params=config,
        dtrain=train_set,
        evals_result=evals_result,
        evals=[(train_set, "train")],
        verbose_eval=False,
        ray_params=ray_params)
    bst.save_model("model.xgb")

from ray import tune

# Specify the hyperparameter search space.
config = {
    "tree_method": "approx",
    "objective": "binary:logistic",
    "eval_metric": ["logloss", "error"],
    "eta": tune.loguniform(1e-4, 1e-1),
    "subsample": tune.uniform(0.5, 1.0),
    "max_depth": tune.randint(1, 9)
}

# Make sure to use the `get_tune_resources` method to set the `resources_per_trial`
analysis = tune.run(
    train_model,
    config=config,
    metric="train-error",
    mode="min",
    num_samples=4,
    resources_per_trial=ray_params.get_tune_resources())
print("Best hyperparameters", analysis.best_config)

2021-11-16 09:40:51,699	WARNING function_runner.py:564 -- Function checkpointing is disabled. This may result in unexpected behavior when using checkpointing features or certain schedulers. To enable, set the train function arguments to be `func(config, checkpoint_dir=None)`.
2021-11-16 09:40:51,719	INFO logger.py:606 -- pip install 'ray[tune]' to see TensorBoard files.
2021-11-16 09:40:51,724	WARNING callback.py:115 -- The TensorboardX logger cannot be instantiated because either TensorboardX or one of it's dependencies is not installed. Please make sure you have the latest version of TensorboardX installed: `pip install -U tensorboardx`


Trial name,status,loc,eta,max_depth,subsample
train_model_497cc_00000,PENDING,,0.000452172,4,0.835778
train_model_497cc_00001,PENDING,,0.000152947,8,0.722635
train_model_497cc_00002,PENDING,,0.000631067,7,0.537654
train_model_497cc_00003,PENDING,,0.000511171,6,0.681208


(raylet) /usr/local/lib/python3.7/dist-packages/redis/connection.py:77: UserWarning: redis-py works best with hiredis. Please consider installing
(raylet)   warnings.warn(msg)
(pid=397) /usr/local/lib/python3.7/dist-packages/xgboost_ray/main.py:166: UserWarning: You are using `xgboost_ray` with a legacy XGBoost version (version 0.90). While we try to support older XGBoost versions, please note that this library is only fully tested and supported for XGBoost >= 1.4. Please consider upgrading your XGBoost version (`pip install -U xgboost`).
(pid=397)   warnings.warn(LEGACY_WARNING)
(pid=397) /usr/local/lib/python3.7/dist-packages/xgboost_ray/main.py:422: UserWarning: `num_actors` in `ray_params` is smaller than 2 (1). XGBoost will NOT be distributed!
(pid=397)   f"`num_actors` in `ray_params` is smaller than 2 "
(pid=397) 2021-11-16 09:40:54,383	INFO main.py:971 -- [RayXGBoost] Created 1 new actors (1 total actors). Waiting until actors are ready for training.
(raylet) /usr/local/lib/pyt

Trial name,status,loc,eta,max_depth,subsample
train_model_497cc_00000,RUNNING,172.28.0.2:397,0.000452172,4,0.835778
train_model_497cc_00001,PENDING,,0.000152947,8,0.722635
train_model_497cc_00002,PENDING,,0.000631067,7,0.537654
train_model_497cc_00003,PENDING,,0.000511171,6,0.681208


(pid=397) 2021-11-16 09:40:57,300	INFO main.py:1016 -- [RayXGBoost] Starting XGBoost training.


...

Result for train_model_497cc_00003:
  date: 2021-11-16_09-41-18
  done: false
  experiment_id: 4b79494bf980400f92b1f322a4adcb25
  hostname: 17bb4599e5e6
  iterations_since_restore: 1
  node_ip: 172.28.0.2
  pid: 711
  time_since_restore: 4.26694917678833
  time_this_iter_s: 4.26694917678833
  time_total_s: 4.26694917678833
  timestamp: 1637055678
  timesteps_since_restore: 0
  train-error: 0.040422
  train-logloss: 0.692711
  training_iteration: 1
  trial_id: 497cc_00003
  
Result for train_model_497cc_00003:
  date: 2021-11-16_09-41-18
  done: true
  experiment_id: 4b79494bf980400f92b1f322a4adcb25
  experiment_tag: 3_eta=0.00051117,max_depth=6,subsample=0.68121
  hostname: 17bb4599e5e6
  iterations_since_restore: 10
  node_ip: 172.28.0.2
  pid: 711
  time_since_restore: 4.548130512237549
  time_this_iter_s: 0.007172346115112305
  time_total_s: 4.548130512237549
  timestamp: 1637055678
  timesteps_since_restore: 0
  train-error: 0.017575
  train-logloss: 0.688772
  training_iteration: 

(pid=711) 2021-11-16 09:41:18,700	INFO main.py:1498 -- [RayXGBoost] Finished XGBoost training on training data with total N=569 in 4.54 seconds (1.35 pure XGBoost training time).


Trial name,status,loc,eta,max_depth,subsample,iter,total time (s),train-logloss,train-error
train_model_497cc_00000,TERMINATED,172.28.0.2:397,0.000452172,4,0.835778,10,4.28008,0.689224,0.01406
train_model_497cc_00001,TERMINATED,172.28.0.2:501,0.000152947,8,0.722635,10,4.51835,0.69184,0.022847
train_model_497cc_00002,TERMINATED,172.28.0.2:605,0.000631067,7,0.537654,10,4.30207,0.687843,0.022847
train_model_497cc_00003,TERMINATED,172.28.0.2:711,0.000511171,6,0.681208,10,4.54813,0.688772,0.017575


2021-11-16 09:41:18,872	INFO tune.py:630 -- Total run time: 27.18 seconds (26.98 seconds for the tuning loop).


Best hyperparameters {'tree_method': 'approx', 'objective': 'binary:logistic', 'eval_metric': ['logloss', 'error'], 'eta': 0.0004521719682812445, 'subsample': 0.8357778803882938, 'max_depth': 4}
